1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los      documentos.
   Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad    según el contenido del texto y la etiqueta de clasificación.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
     

In [5]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()


In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]
     

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# Tomar 5 documentos al azar
idxs = np.random.choice(len(newsgroups_train.data), size=5, replace=False)

for idx in idxs:
    # Obtener el documento
    doc = newsgroups_train.data[idx]
    
    # Vectorizar el documento
    doc_vec = tfidfvect.transform([doc])
    
    # Calcular similitud coseno con todos los documentos
    cossim = cosine_similarity(doc_vec, X_train)[0]
    
    # Ordenar los índices por similitud descendente
    mostsim = np.argsort(cossim)[::-1][1:6]
    
    # Imprimir el documento original y sus 5 más similares
    print(f"Documento original ({newsgroups_train.target_names[y_train[idx]]}): {doc}")
    print("Más similares:")
    for i in mostsim:
        print(f"- {newsgroups_train.data[i]} ({newsgroups_train.target_names[y_train[i]]})")
    print()

Documento original (sci.crypt): 
Más similares:
- 

Again, from my alcohol server's class:
The absolute *most* that eating before drinking can do is slow the absorption
down by 15 minutes.  That gives me time to eat, slam one beer, and ride like
hell to try to make it home in the 10 minutes left after paying, donning 
helmet & gloves, starting bike...

 (rec.motorcycles)
- I will try to answer some of Dorin's questions, even though they were
not addressed to me specifically, but I feel that I am a bit concerned
by the thread since I am a Southern Lebanese from a village that is 
often on the receiving end of Israel's bombs.
In the first place the death of three soldiers on a patrol in occupied
Lebanese terrritory is NOT an act of terrorism or murder.  It is 
disingeneous to compare their death to that of athletes in Munich
or any other act of terrorism or mrder.  This exercise is aimed 
solely at diverting the issue and is far from the truth.
It seems to me, Dorin, that, you are so rem

2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [17]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [18]:
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [19]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6068927614955861

In [20]:
# Probar diferentes parámetros del vectorizador
params = {
    'max_df': [0.5, 0.7, 0.9],
    'min_df': [2, 5, 10],
    'max_features': [5000, 10000, 20000]
}

best_f1 = 0
best_params = None
best_model = None

for max_df in params['max_df']:
    for min_df in params['min_df']:
        for max_features in params['max_features']:
            # Instanciar el vectorizador con los parámetros actuales
            tfidfvect = TfidfVectorizer(max_df=max_df, min_df=min_df, max_features=max_features)
            
            # Vectorizar los datos de entrenamiento
            X_train = tfidfvect.fit_transform(newsgroups_train.data)
            
            # Probar ambos modelos Naïve Bayes
            for model in [MultinomialNB(), ComplementNB()]:
                model.fit(X_train, y_train)
                
                # Vectorizar los datos de test
                X_test = tfidfvect.transform(newsgroups_test.data)
                
                # Predecir etiquetas en test
                y_pred = model.predict(X_test)
                
                # Calcular F1-score macro
                f1 = f1_score(y_test, y_pred, average='macro')
                
                # Actualizar el mejor modelo si es necesario
                if f1 > best_f1:
                    best_f1 = f1
                    best_params = {'max_df': max_df, 'min_df': min_df, 'max_features': max_features}
                    best_model = model
                    
print(f"Mejor F1-score macro: {best_f1:.3f}")
print(f"Mejores parámetros: {best_params}")
print(f"Mejor modelo: {type(best_model).__name__}")

Mejor F1-score macro: 0.688
Mejores parámetros: {'max_df': 0.9, 'min_df': 2, 'max_features': 20000}
Mejor modelo: ComplementNB


3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [21]:
# Transponer la matriz documento-término
X_train_T = X_train.T

# Tomar 5 palabras "manualmente"
words = ['computer', 'science', 'machine', 'learning', 'artificial']

for word in words:
    # Obtener el índice de la palabra en el vocabulario
    word_idx = tfidfvect.vocabulary_[word]
    
    # Obtener el vector de la palabra
    word_vec = X_train_T[word_idx]
    
    # Calcular similitud coseno con todos los vectores de palabras
    cossim = cosine_similarity(word_vec, X_train_T)[0]
    
    # Ordenar los índices por similitud descendente
    mostsim = np.argsort(cossim)[::-1][1:6]
    
    # Imprimir la palabra y sus 5 más similares
    print(f"Palabra: {word}")
    print("Más similares:")
    for i in mostsim:
        # Obtener la palabra a partir del índice
        similar_word = idx2word[i]
        print(f"- {similar_word}")
    print()

Palabra: computer
Más similares:
- 6fpys
- 0ja
- 5uj9z
- 60061
- 3cm

Palabra: science
Más similares:
- 4yr8kyli
- 1c
- 506
- 25ah
- 1rh8zrck

Palabra: machine
Más similares:
- 01001010
- 0ll
- 047
- 6ei0mfq
- 4t164obex3625

Palabra: learning
Más similares:
- 2mg
- 1500
- 1024
- 176
- 34f9z_f

Palabra: artificial
Más similares:
- 0d
- 1np6c
- 10100010
- 1alk
- 1018

